# 실습: CLIP과 GPT-2를 이용한 Zero-Shot 이미지 캡셔닝

본 실습의 목표는 Sub PJT 2에서 실습한 CLIP과 GPT-2를 사용하여 학습이 필요 없는 optimization 기반의 zero-shot 이미지 캡셔닝 구현입니다. 
**구현시 막히는 부분이 생기면 Sub PJT 2의 CLIP과 GPT-2의 구현 내용들을 적극 참고해서 실습을 진행합니다.** 
먼저 CLIP과 GPT-2에 대한 간단한 복습을 한 뒤 실습을 합니다.

# CLIP: connecting text and images, by OpenAI

![clip](images/CLIP.png)

- Natural language supervision으로부터 visual concepts을 학습하는 방법
- Image encoder: ViT-B (or ResNet50)
- Text encoder: Transformers

### Contrastive Learning
![con](images/Contrastive.png)
- CLIP은 Contrastive learning 방식으로 학습
- Target image(anchor)와 matching image (positive)는 가까워지도록 학습
- Anchor와 많은 non-matching images (negative)는 멀어지도록 학습

# Background of GPT-2

![gpt2](images/gpt-2.png)

- 새로운 next token은 last token과 이전의 key-values(context)에 의해 예측된다
- `Context`는 Transformer layer로부터 나온 key-value pairs를 의미한다

# ZeroCap: Zero-Shot Image-to-Text Generation for Visual-Semantic Arithmetic

본 실습에서 구현할 예정인 "ZeroCap" 모델에 대한 설명 입니다.

### 모델 설명

![zerocap](images/zerocap.png)


- 주어진 image에 대해 설명하는 text를 생성해내는 모델으로, 추가적인 학습 과정이 필요 없고 오직 optimization만 필요
- Large-scale language model(GPT-2)을 large-scale text-image alignment(CLIP)을 사용하여 가이드하는 zero-shot method 
- 이전 방법들보다 더 다양하고 less scripted한 text generation method


필요한 package와 dependency들을 설치해줍니다.

CLIP과 GPT-2를 활용하기 위한 transformers를 설치해주고, 이를 활용하여 imate-to-text generation을 진행할 zero-shot-image-to-text (ZeroCap) 을 설치해줍니다.

In [ ]:
!pip install attrdict
!pip install git+https://github.com/openai/CLIP.git
!pip install transformers

In [ ]:
!mv /content/zero-shot-image-to-text /content/zero_shot_image_to_text

[ZeroCap: Zero-Shot Image-to-Text Generation for Visual-Semantic Arithmetic] 논문에서 사용된 hyper-parameters 세팅을 설정해줍니다.

In [ ]:
from attrdict import AttrDict
from enum import Enum
import torch
import clip
import numpy as np
from IPython.display import Image, display
from PIL import Image as PIL_Image

class GenType(Enum):
    Captioning = 0
    Arithmetic = 1
    RealWorld = 2
    OCR = 3
    
def get_args(gen_type):
    args = {}
    args['lm_model']="gpt-2"  # language model로 transformer-based LM인 GPT-2 사용
    args['forbidden_tokens_file_path']='files/forbidden_tokens.npy'
    args['target_seq_length']=15
    args['reset_context_delta']=True
    args['num_iterations']=5
    args['clip_loss_temperature']=0.01
    args['clip_scale']=1
    args['ce_scale']=0.2
    args['stepsize']=0.3
    args['repetition_penalty']=1
    args['end_token']="."
    args['forbidden_factor']=20
    args['beam_size']=5

    if gen_type == GenType.Captioning:
        args['cond_text']="Image of a"  # Initial propmpt 값으로, GPT-2에서 next word prediction을 하기 위한 시작 문장
        args['end_factor']=1.01
        args['fusion_factor']=0.99
        args['grad_norm_factor']=0.9
        
    return AttrDict(args)

### CLIPTextGenerators 클래스 구현
본 실습은 CLIPTextGenerators 클래스의 함수들을 하나씩 구현하면서 Req. 1-x (1-3 제외)를 푸는게 목표입니다. 구현에 필요한 부분은 다음과 같습니다. 아래 셀부터 하나씩 따라가면서 Req. 1-x에서 구현이 필요한 부분을 읽고, 다시 CLIPTextGenerators 함수 정의로 돌아와서 해당 부분을 구현해봅니다. 

**Req. 1-1:** CLIPTextGenerators 내에 get_img_feature() 메소드 구현

**Req. 1-2:** CLIPTextGenerators 내에 get_txt_features() 메소드 구현

**Req. 1-3:** 생성된 캡션 중 이미지와 가장 비슷한 캡션 선정

**Req. 1-4:** CLIPTextGenerators 내에 print_captions() 메소드 구현. 

**Req. 1-5:** CLIPTextGenerators 내에 clip_loss() 메소드 구현

**Req. 1-6:** CLIPTextGenerators 내에 ce_loss() 메소드 구현

In [ ]:
from torch import nn

from zerocap import CLIPTextGenerator
from zerocap import add_context

class CLIPTextGenerators(CLIPTextGenerator):
    ############################################################################
    # Req 1-1: CLIPTextGenerators 내에 get_img_feature() 메소드 구현.              #
    ############################################################################
    def get_img_feature(self, img_path):

      imgs = [PIL_Image.open(x) for x in img_path]
      image_features=torch.zeros((1,1)).cuda()
      ################################################################################
      # TODO: img_path를 입력받아 CLIP의 image encoder를 통해 image feature 출력.          #
      ################################################################################
      # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
      # 1) CLIP에서 정의된 preprocess 함수로 imgs preprocess
      # 2) CLIP의 image encoder를 사용하여 image_features 추출
      # 3) Norm=1 인 벡터로 normalize

      pass

      # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
      ################################################################################
      #                                 END OF YOUR CODE                             #
      ################################################################################

      return image_features.detach()
    
    ############################################################################
    # Req 1-2: CLIPTextGenerators 내에 get_txt_features() 메소드 구현.              #
    ############################################################################
    def get_txt_features(self, text):
        text_features=torch.zeros((1,1)).cuda()
        ################################################################################
        # TODO: text 리스트를 입력받아 CLIP의 text encoder를 통해 text features 출력
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) CLIP에서 정의된 tokenize 함수로 text를 token으로 변경
        # 2) CLIP의 text encoder를 사용하여 text_features 추출
        # 3) Norm=1 인 벡터로 normalize

        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

        return text_features.detach()
      
    ############################################################################
    # Req 1-4: CLIPTextGenerators 내에 print_captions() 메소드 구현.           #
    ############################################################################
    def print_captions(self, scores, gen_tokens, seq_lengths):
        output_texts=None
        ################################################################################
        # TODO: 생성된 token을 text로 변환하여 출력
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) 각 캡션별 score의 평균값 계산 (scores/seq_lengths)
        # 2) 생성된 token (gen_tokens)을 text로 변환 (GPT-2 tokenizer 사용)
        # 3) score이 높은 순으로 캡션을 나열하여 반환

        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

        return output_texts      


    ############################################################################
    # Req 1-5: CLIPTextGenerators 내에 clip_loss() 메소드 구현.                    #
    ############################################################################  
    def clip_loss(self, probs, context_tokens):        
        for p_ in self.clip.transformer.parameters():
            if p_.grad is not None:
                p_.grad.data.zero_()

        # 각 캡션에서 예측한 다음 단어 중 Top 512 token candidates 추출
        top_size =512
        _, top_indices = probs.topk(top_size, -1)
        
        # CLIP loss 누적, 각 캡션 별 loss 값 append
        clip_loss = 0
        losses = []
        
        ################################################################################
        # TODO: 512개의 token 후보와 image feature와 cosine 비교후 loss값 계산
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # context_tokens ([beam_size, x]) : beam_size개의 캡션의 context_token
        # probs ([beam_size, 50257]) : context_token을 사용하여 GPT-2에서 예측한 다음 단어의 probability
        #                      beam_size개의 캡션이 각각 50257개의 단어의 probability 출력
        # 1) context_tokens를 text로 decoding (GPT-2 decode 메소드 활용)
        # 2) For문으로 beam_size개의 캡션 별로 다음 내용을 수행
        # 3) Top 512 token으로 후보 문장 512개 생성 (GPT-2 decode 메소드 활용) 후 리스트에 저장
        # 4) 3)에서 얻은 512개 텍스트의 text feature (self.get_txt_features() 메소드 활용) 추출
        # 5) image feature와 text feature 간 cosine distance 계산
        # 6) 앞서 구한 cosine distance와 probability 간의 cross-entropy loss 계산
        # 7) clip_loss에 loss 누적, losses 리스트에 loss 값 append
        # 8) 5개 캡션에 대해 3)~7) 반복 

        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

        return clip_loss, losses
    
    ############################################################################
    # Req 1-6: CLIPTextGenerators 내에 ce_loss() 메소드 구현.                    #
    ############################################################################
    def ce_loss(self, shift_prob, before_shift_prob):
        ################################################################################
        # TODO: 기존 GPT-2의 context로 예측한 다음 단어의 probability (before_shift_prob)과   #
        # update된 context로 예측한 다음 단어의 probability (shitf_prob) 간의                 #
        # cross-entropy loss 측정                                                       #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) shift_prob와 before_shift_prob간의 cross-entropy loss를 측정
        # 2) 이 때 self.ce_scale 값을 cross_entropy loss에 곱해줘서 최종 ce_loss 측정
        ce_loss=None
        pass
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

        return ce_loss
    
    def generate_captions(self, image_features, cond_text, beam_size):
      # 입력 이미지의 image feature (CLIP embedding)
      self.image_features = image_features

      # 시작 context_token 정의
      # 본 실습의 시작 context token: (self.context_prefix)+ "Image of a" (cond_text)
      # [1,4] shape
      context_tokens = self.lm_tokenizer.encode(self.context_prefix + cond_text)

      # 앞에서 계산한 context_tokens으로 GPT-2에 전달해 text generation
      # self.generate_text() 내부에서 clip_loss, ce_loss 계산하여 output_text 생성
      output_tokens, output_text = self.generate_text(context_tokens, beam_size)

      return output_text
    
    def save_image_features(self, image_features):
      self.image_features = image_features

## CLIPTextGenerator의 메소드 구현

본 실습에서는 먼저 CLIPTextGenerators 내의 image/text feature를 출력하는 메소드를 각각 구현합니다. Image와 text의 feature를 뽑기 위해서는 CLIP의 pretrained encoder들을 사용합니다.

### Req. 1-1:	CLIPTextGenerators 내에 get_img_feature() 메소드 구현

### Req. 1-2:	CLIPTextGenerators 내에 get_txt_features() 메소드 구현

**아래 cell을 실행 한 뒤 similarity:=tensor([[0.2793],[0.1481],[0.1403]]) 가 나오면 성공.**

In [ ]:
# 예시 이미지
img_path = 'example/example.jpg'

# CLIPTextGenerators 인스턴스 생성 
captioning_args = get_args(GenType.Captioning)
text_generator = CLIPTextGenerators(**dict(captioning_args))

# input image에 대한 CLIP image feature 추출
image_features = text_generator.get_img_feature([img_path])

# input image에 대한 CLIP image feature 추출
text=["White bathroom", "Green tree", "Blue car"]
text_features = text_generator.get_txt_features(text)

# image feature와 text features 간의 cosine similarity 계산
print("similarity: ",text_features@image_features.T)

### Req. 1-3:	생성된 캡션 중 이미지와 가장 비슷한 캡션 선정

본 실습에서는 생성된 캡션들과 image feature를 입력받아, 이미지와 가장 맞는 캡션을 출력하는 함수를 구현합니다.

In [ ]:
# 입력 이미지의 CLIP embedding (feature) 와 가장 잘 맞는 (most aligned) caption을 return하는 코드
# Alignment score는 text와 image CLIP features의 cosine distance로 계산

def calc_best_clip(text_generator, captions, image_feature):
    ################################################################################
    # TODO: 생성된 캡션 중 이미지와 가장 비슷한 캡션 선정.                                   #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    # 1) text_generator의 encode_text() 메소드를 사용하여 생성된 caption들의 text features 추출합니다.
    # 2) text features와 image feature 사이의 cosine similarity를 계산합니다.
    # 3) cosine similarity 값이 가장 큰 캡션을 선정합니다.
    best_clip=None
    pass

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################
    return best_clip

In [ ]:
print('Input image:')
display(Image(img_path, width = 300, height = 300))

best_clip = calc_best_clip(text_generator, text, image_features)
print("best clip: ",best_clip)


### Req. 1-4: CLIPTextGenerators 내에 print_captions() 메소드 구현.
생성한 token을 입력받아서 텍스트로 변환하는 부분을 구현합니다.

**아래 cell을 실행 한 뒤 output=['�', ' ind', 'inburgh', ' Sor', 'TF'] 가 나오면 성공.**

In [ ]:
torch.manual_seed(123)

# CLIPTextGenerators 인스턴스 생성 
captioning_args = get_args(GenType.Captioning)
text_generator = CLIPTextGenerators(**dict(captioning_args))

scores_temp = torch.Tensor([-1.3940, -2.7275, -3.5372, -3.5392, -3.9981]).cuda()
gen_tokens_temp = torch.Tensor([[123],[773],[22222],[15423],[10234]]).cuda()
seq_lengths_temp = torch.ones((5)).cuda()

output=text_generator.print_captions(scores_temp,gen_tokens_temp,seq_lengths_temp)
print("output:",output)

## Loss function 구현

이미지 캡셔닝 구현을 위한 loss function을 구현합니다. 앞서 언급한 대로, 본 실습에서는 별도의 학습 과정은 없습니다. 대신 CLIP loss와 CE loss를 사용하여 GPT-2의 context (key, value) 값을 업데이트 해 주어 최종 캡션을 생성할 수 있습니다.





### ZeroCap Loss Function

![loss](images/loss_func.png)


- CLIP loss: 입력 이미지의 CLIP embedding과 생성된 text의 CLIP embedding의 cosine distance를 사용하여 주어진 image를 서술하는 sentences를 생성할 수 있도록 한다
- CE loss: 새롭게 생성되어 업데이트 된 words들의 probability distribution이 original LM과 비슷해지도록 하여 생성된 문장의 유창성을 보장한다
- GPT-2의 context와 last token으로 original token generation을 진행하고, CLIP loss 기반으로 context를 업데이트하여 shifted token을 생성하여, image를 잘 describe하는 text 생성
- Original token과 shifted token 사이 CE loss를 통해 language fluency 보장


### Req. 1-5: CLIPTextGenerators 내에 clip_loss() 메소드 구현
**아래 cell을 실행 한 뒤 clip_loss=43.3372, losses[0]=8.6827 가 나오면 성공.**

In [ ]:
import torch.nn as nn
torch.manual_seed(123)
softmax=nn.Softmax(dim=1)

# 예시 이미지
img_path = 'example/example.jpg'

# CLIPTextGenerators 인스턴스 생성 
captioning_args = get_args(GenType.Captioning)
text_generator = CLIPTextGenerators(**dict(captioning_args))

# input image에 대한 CLIP image feature 추출
image_features = text_generator.get_img_feature([img_path])
text_generator.save_image_features(image_features)

temp1 = softmax(torch.randn((5,50257)).cuda())
temp2 = torch.randint(500,(5,5)).cuda()

clip_loss, losses= text_generator.clip_loss(temp1, temp2)
print("clip_loss: ",clip_loss)
print("losses[0]: ",losses[0])

### Req. 1-6: CLIPTextGenerators 내에 ce_loss() 메소드 구현
**아래 cell을 실행 한 뒤 ce_loss[0]=0.2020 가 나오면 성공.**

In [ ]:
import torch.nn as nn
torch.manual_seed(456)
softmax=nn.Softmax(dim=1)

# CLIPTextGenerators 인스턴스 생성 
captioning_args = get_args(GenType.Captioning)
text_generator = CLIPTextGenerators(**dict(captioning_args))

temp1 = softmax(torch.randn((5,50257)).cuda())
temp2 = softmax(torch.randn((5,50257)).cuda())

ce_loss= text_generator.ce_loss(temp1, temp2)
print("ce_loss[0]: ",ce_loss[0])

## 이미지 캡셔닝 실행
앞선 Req. 1-x 를 모두 구현 하면 아래 cell을 실행하여 이미지 캡셔닝을 수행할 수 있습니다.
제공해준 사진 이외의 본인만의 사진을 넣어보며 다양한 결과들을 확인합니다.

In [ ]:
img_path = 'example/example.jpg'

print('Input image:')
display(Image(img_path, width = 300, height = 300))

# CLIPTextGenerators 인스턴스 생성 
captioning_args = get_args(GenType.Captioning)
text_generator = CLIPTextGenerators(**dict(captioning_args))

# input image에 대한 CLIP image feature 추출
image_features = text_generator.get_img_feature([img_path])

captions = text_generator.generate_captions(image_features, captioning_args.cond_text, beam_size=captioning_args.beam_size)


생성된 tokens들에서 best CLIP score를 계산

In [ ]:
best_clip = calc_best_clip(text_generator, captions, image_features)

print('all captions:', captions)
print('best clip:', captioning_args.cond_text + best_clip)